# Ejercicio



Considere la siguiente clase que se llama DateTime, es tiene un timezone, dia, año, mes, hora, minuto y segundo.

In [1]:
import requests

class DateTime:
    def __init__(self, timezone: str):
        self.timezone = timezone
        self._initialize_current_time(timezone)

    def _initialize_current_time(self, timezone: str):
        datetime_str = self.request_time(timezone)

        date, time = datetime_str.split('T')
        self.year, self.month, self.day = map(int, date.split('-'))
        time_parts = time.split(':')
        self.hour = int(time_parts[0])
        self.minute = int(time_parts[1])

    def request_time(self,timezone: str):
        url = f"http://worldtimeapi.org/api/timezone/{timezone}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        datetime_str = data['datetime'][:-1]  # Removing the 'Z' for ISO format
        return datetime_str

    def __str__(self):
        return f"{self.year:04}-{self.month:02}-{self.day:02} {self.hour:02}:{self.minute:02}"


In [2]:
londonTime= DateTime("America/Santiago")
print(londonTime)

2024-08-27 12:30


In [3]:
class Appointment:
    def __init__(self, year: int, month: int, day: int, hour: int, minute: int, timezone: str):
        self.appointment_time = DateTime(timezone)
        self.appointment_time.year = year
        self.appointment_time.month = month
        self.appointment_time.day = day
        self.appointment_time.hour = hour
        self.appointment_time.minute = minute
        self.appointment_time.second = 0  # Assuming seconds are zero for appointments

    def is_late(self) -> bool:
        current_time = DateTime(self.appointment_time.timezone)
        if (current_time.year > self.appointment_time.year or
            (current_time.year == self.appointment_time.year and current_time.month > self.appointment_time.month) or
            (current_time.year == self.appointment_time.year and current_time.month == self.appointment_time.month and current_time.day > self.appointment_time.day) or
            (current_time.year == self.appointment_time.year and current_time.month == self.appointment_time.month and current_time.day == self.appointment_time.day and current_time.hour > self.appointment_time.hour) or
            (current_time.year == self.appointment_time.year and current_time.month == self.appointment_time.month and current_time.day == self.appointment_time.day and current_time.hour == self.appointment_time.hour and current_time.minute > self.appointment_time.minute)):
            return True
        return False


In [4]:
appointment = Appointment(2024, 8, 8, 17, 00, "America/Santiago")
print(f"Appointment Time: {appointment.appointment_time}")
if appointment.is_late():
  print("You are late for the appointment.")
else:
  print("You are on time for the appointment.")

Appointment Time: 2024-08-08 17:00
You are late for the appointment.


Un estudiante de Testing hizo este test, el 7 de Agosto del 2024, a las 15:20, y los dos test pasaban. Sin embargo, el profesor reviso su tarea hoy 8 de agosto y uno de los dos test no pasa.

In [5]:
import unittest

class TestAppointment(unittest.TestCase):

  def testOnTime(self):
    a1 = Appointment(2024, 8, 7, 15, 30, "America/Santiago")
    self.assertFalse(a1.is_late())

  def testLate(self):
    a1 = Appointment(2024, 8, 7, 15, 10, "America/Santiago")
    self.assertTrue(a1.is_late())


def runTests(testClass):
  # Create a test suite
  suite = unittest.TestSuite()
  # Add tests from the test class
  suite.addTest(unittest.makeSuite(testClass))
  # Run the test suite
  runner = unittest.TextTestRunner()
  runner.run(suite)

runTests(TestAppointment)

.F
FAIL: testOnTime (__main__.TestAppointment)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\maxim\AppData\Local\Temp\ipykernel_26452\1225853485.py", line 7, in testOnTime
    self.assertFalse(a1.is_late())
AssertionError: True is not false

----------------------------------------------------------------------
Ran 2 tests in 1.248s

FAILED (failures=1)


El problema es la dependencia al API que devulve la hora actual.

**Ejercicio** Ayude al estudiante a que sus test vuelvan a pasar, para esto utilize mocks para que el API devuelva la hora que el estudiante hizo los tests. Escriba la solución abajo:



In [7]:
from unittest.mock import MagicMock
from unittest.mock import patch

import unittest

class TestAppointment(unittest.TestCase):

  @patch.object(DateTime, 'request_time', return_value="2024-08-07T15:20:000000-04:00")
  def testOnTime(self, MockDateTime):
    a1 = Appointment(2024, 8, 7, 15, 30, "America/Santiago")
    self.assertFalse(a1.is_late())

  @patch.object(DateTime, 'request_time', return_value="2024-08-07T15:20:000000-04:00")
  def testLate(self, MockDateTime):
    a1 = Appointment(2024, 8, 7, 15, 10, "America/Santiago")
    self.assertTrue(a1.is_late())


def runTests(testClass):
  suite = unittest.TestSuite()
  suite.addTest(unittest.makeSuite(testClass))
  runner = unittest.TextTestRunner()
  runner.run(suite)

runTests(TestAppointment)


..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK
